In [1]:
import os
import shutil
import math

import torch
import torchvision
import tarfile
from torchvision.datasets.utils import download_url
from torch.utils.data import random_split

import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
matplotlib.rcParams['figure.facecolor'] = '#ffffff'



/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.0.1) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [4]:
# from google.colab import drive
# drive.mount('/content/drive')

In [5]:
# import zipfile
# with zipfile.ZipFile('./Dataset.zip', 'r') as zip_ref:
#     zip_ref.extractall('./dataset')

In [6]:
# import zipfile
# with zipfile.ZipFile('./drive/MyDrive/HW2/Dataset.zip', 'r') as zip_ref:
#     zip_ref.extractall('./dataset')

In [2]:
# import torch
# import torch_directml
# device = torch_directml.device()
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

# Assuming that we are on a CUDA machine, this should print a CUDA device:

print(device)

cpu


In [3]:
# # # # Organize data into training and test datasets

data_dir = './dataset'
test_files = []

# # Remove all previous datasets
# isExist = os.path.exists(data_dir + "/train")
# if isExist:
#     shutil.rmtree(data_dir + "/train")
#     os.mkdir(data_dir + "/train")
# else:
#     os.mkdir(data_dir + "/train")
    
# isExist = os.path.exists(data_dir + "/test")
# if isExist:
#     shutil.rmtree(data_dir + "/test")
#     os.mkdir(data_dir + "/test")
# else:
#     os.mkdir(data_dir + "/test")

# for x in range(10): # Loop 10 times for each class
#     # split images into 8:2
#     files = os.listdir(data_dir + "/" + str(x))
#     train_files = files[:int(len(files)*0.8)]
#     test_files = files[int(len(files)*0.8):int(len(files))]

#       # copy to new path training
    
#     train_files_new = [None] * len(train_files)
#     path = "./dataset/train" + "/" + str(x)

#     isExist = os.path.exists(data_dir + "/train/" + str(x))
#     os.mkdir(data_dir + "/train/" + str(x))
#     if isExist:
#       shutil.rmtree(data_dir + "/train/" + str(x))
#       os.mkdir(data_dir + "/train/" + str(x))
        

#     for y in range(len(train_files)):
#         print(data_dir + "/" + train_files[y])
#         shutil.copyfile(data_dir + "/" + str(x) + "/" + train_files[y], path + "/" + train_files[y])


    
#     # copy to new path test
    
#     isExist = os.path.exists(data_dir + "/test/" + str(x))
#     os.mkdir(data_dir + "/test/" + str(x))
#     if isExist:
#       shutil.rmtree(data_dir + "/test/" + str(x))
#       os.mkdir(data_dir + "/test/" + str(x))

#     test_files_new = [None] * len(test_files)
#     path = "./dataset/test" + "/" + str(x)

#     for y in range(len(test_files)):
#         print(data_dir + "/" + test_files[y])
#         shutil.copyfile(data_dir + "/" + str(x) + "/" + test_files[y], path + "/" + test_files[y])
    
  
        

In [4]:
from torchvision.datasets import ImageFolder
from torchvision.transforms import ToTensor
from avalanche.benchmarks import nc_benchmark
from avalanche.benchmarks.utils import make_classification_dataset
from avalanche.benchmarks.generators import filelist_benchmark, dataset_benchmark

train_dataset = ImageFolder(data_dir+'/train', transform=ToTensor())
test_dataset = ImageFolder(data_dir+'/test', transform=ToTensor())
print(type(train_dataset))

scenario_custom_task_labels = dataset_benchmark(
    [train_dataset,train_dataset]
 
   ,[test_dataset,test_dataset]
)
train_stream = train_dataset
test_stream = test_dataset



/home/juwonpee/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'torchvision.datasets.folder.ImageFolder'>


In [5]:
# define CNN

import torch.nn as nn
import torch
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=False),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=False),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc1 = nn.Linear(32*16*16, 2048)
        self.fc2 = nn.Linear(2048, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 32)
        self.fc5 = nn.Linear(32, 10)
            
    def forward(self, x):
        x = self.conv(x)

        x = x.view(-1, 32*16*16)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.fc2(x)
        x = torch.relu(x)
        x = self.fc3(x)
        x = torch.relu(x)
        x = self.fc4(x)
        x = torch.relu(x)
        x = self.fc5(x)
        x = torch.log_softmax(x, dim=1)
        return x

In [7]:
from avalanche.benchmarks.classic import SplitCIFAR10
from avalanche.benchmarks.generators import nc_benchmark
from avalanche.models import SimpleMLP, as_multitask
from avalanche.training.supervised import Naive, LwF, GDumb, AGEM, EWC, JointTraining
import torch.optim as optim
from torch.nn import CrossEntropyLoss
from torch.optim import Adam
from torch.utils.data import DataLoader
from avalanche.training.plugins import ReplayPlugin, CoPEPlugin, EvaluationPlugin


# model = SimpleMLP(input_size=32 * 32 * 3, num_classes=10)
# model = as_multitask(model, "classifier")
model = CNN()


    # Prepare for training & testing
optimizer = Adam(model.parameters(), lr=0.01)
criterion = CrossEntropyLoss()


benchmark = nc_benchmark(train_dataset=train_stream, test_dataset=test_stream, n_experiences=5,task_labels=True, class_ids_from_zero_in_each_exp=True)


# benchmark = SplitCIFAR10(
#         n_experiences=5,
#         return_task_id=True,
#         class_ids_from_zero_in_each_exp=True,
# )  

# Choose a CL strategy
# LwF
# strategy = LwF(
#         model=model,
#         optimizer=optimizer,
#         criterion=criterion,
#         alpha=1,
#         temperature=1,
#         train_epochs=3,
#         train_mb_size=128,
#         eval_mb_size=128,
#         device=device,
#         plugins=[ReplayPlugin(mem_size=10000)],
#     )

# # Naive with replay
# strategy = Naive(
#         model=model,
#         optimizer=optimizer,
#         criterion=criterion,
#         train_mb_size=128,
#         train_epochs=10,
#         eval_mb_size=128,
#         device=device,
#         plugins=[ReplayPlugin(mem_size=10000)],
#     )


# AGEM
# from avalanche.training.plugins import EvaluationPlugin
# from avalanche.logging import InteractiveLogger
# from avalanche.evaluation.metrics import (
#     forgetting_metrics,
#     accuracy_metrics,
#     loss_metrics,
# )
# interactive_logger = InteractiveLogger()
# eval_plugin = EvaluationPlugin(
#       accuracy_metrics(
#           minibatch=True, epoch=True, experience=True, stream=True
#       ),
#       loss_metrics(minibatch=True, epoch=True, experience=True, stream=True),
#       forgetting_metrics(experience=True),
#       loggers=[interactive_logger],
# )

# strategy = AGEM(
#             model,
#             optimizer,
#             criterion,
#             256, #args.patterns_per_exp,
#             256, #args.sample_size,
#             train_epochs= 5,#args.epochs,
#             device=device,
#             train_mb_size=128,
#             eval_mb_size=128,
#             evaluator=eval_plugin,
#         )

# strategy = JointTraining(
#         model=model,
#         optimizer=optimizer,
#         criterion=criterion,
#         train_mb_size=128,
#         train_epochs=5,
#         eval_mb_size=128,
#         device=device,
#     )
# strategy.train(benchmark.train_stream)
# strategy.eval(benchmark.test_stream)

# # EWC
# strategy = EWC(
#         model=model,
#         optimizer=optimizer,
#         criterion=criterion,
#         0.4, #args.ewc_lambda,
#         "separate" #args.ewc_mode,
#         decay_factor=0.1
#         train_epochs=1,
#         device=device,
#         train_mb_size=128,
#         eval_mb_size=128
          # mem_size=50000,
#     )

# # GDumb
strategy = GDumb(
        model=model,
        optimizer=optimizer,
        criterion=criterion,
        train_mb_size=128,
        eval_mb_size=128,
        train_epochs=1,
        eval_every=0,
        device=device,
        mem_size=50000,
)

# train and test loop
for experience in benchmark.train_stream:
        print("Start of experience ", experience.current_experience)
        strategy.train(experience)
        
        print("Training completed")

        print("Computing accuracy on the whole test set")
        strategy.eval(benchmark.test_stream)

Start of experience  0
-- >> Start of training phase << --
  6%|▋         | 51/800 [01:27<13:44,  1.10s/it]

KeyboardInterrupt: 